In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
print("libraries imported")

libraries imported


In [6]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [7]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [8]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [14]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'CIHOMOGBEYXETSJ2AYK2J41YQCMQUQH2ADGX1LUUC0PQSUH2'
CLIENT_SECRET = 'EBRDDJ43CTYVR103OHLZLGDONUSFFDKNW1OP2K3GQQVMUAVP'
VERSION = '20181020'

In [15]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_cafe = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16d941735')
newyork_venues_cafe.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Gold Mine Cafe,40.878916,-73.904698,Café
1,Marble Hill,40.876551,-73.91066,Sugarboy Bakery Cafe,40.877948,-73.902860,Bakery
2,Marble Hill,40.876551,-73.91066,Amores Cafe & Salad Juice Bar,40.869485,-73.916754,Café
3,Marble Hill,40.876551,-73.91066,JH Café,40.868210,-73.902124,Café
4,Marble Hill,40.876551,-73.91066,Moss Café: Farm-To-Table Restaurant and Coffee...,40.885269,-73.910103,Café


In [17]:
newyork_venues_cafe.shape

(1763, 7)

In [18]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [21]:
map_newyork_cafe = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_cafe, 'red', map_newyork_cafe)

map_newyork_cafe

In [25]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [24]:
manhattan_grouped = newyork_venues_cafe.groupby('Neighborhood').count()
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,50,50,50,50,50,50
Carnegie Hill,50,50,50,50,50,50
Central Harlem,32,32,32,32,32,32
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
East Harlem,39,39,39,39,39,39
East Village,50,50,50,50,50,50


In [26]:
manhattan_onehot = pd.get_dummies(newyork_venues_cafe[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_cafe['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Australian Restaurant,Austrian Restaurant,Bagel Shop,Bakery,Bar,Beer Garden,Bookstore,Breakfast Spot,Bubble Tea Shop,Cafeteria,Café,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cooking School,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Ethiopian Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Gym / Fitness Center,Hotel Bar,Ice Cream Shop,Internet Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Pet Café,Pet Service,Pharmacy,Pizza Place,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Sushi Restaurant,Swiss Restaurant,Tea Room,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Australian Restaurant,Austrian Restaurant,Bagel Shop,Bakery,Bar,Beer Garden,Bookstore,Breakfast Spot,Bubble Tea Shop,Cafeteria,Café,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Cooking School,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Donut Shop,Ethiopian Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Furniture / Home Store,Gaming Cafe,Gym / Fitness Center,Hotel Bar,Ice Cream Shop,Internet Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,New American Restaurant,Pet Café,Pet Service,Pharmacy,Pizza Place,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Sushi Restaurant,Swiss Restaurant,Tea Room,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Battery Park City,0.00,0.000000,0.00,0.00,0.000000,0.080000,0.000000,0.000000,0.00,0.000000,0.04,0.00000,0.460000,0.00,0.120000,0.000000,0.020000,0.00,0.00,0.00,0.060000,0.020000,0.000000,0.020000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.00,0.000000,0.020000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.02,0.020000,0.000000,0.00,0.080000,0.00,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.00
1,Carnegie Hill,0.00,0.020000,0.00,0.02,0.040000,0.100000,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.540000,0.02,0.120000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.020000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.00,0.00,0.02,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.02,0.00,0.000000,0.000000,0.02,0.020000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00
2,Central Harlem,0.00,0.031250,0.00,0.00,0.000000,0.000000,0.031250,0.000000,0.00,0.000000,0.00,0.03125,0.750000,0.00,0.093750,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.031250,0.000000,0.000000,0.03125,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00
3,Chelsea,0.00,0.020000,0.00,0.00,0.000000,0.080000,0.000000,0.000000,0.00,0.000000,0.00,0.00000,0.500000,0.00,0.160000,0.000000,0.000000,0.02,0.00,0.00,0.000000,0.020000,0.040000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.020000,0.02,0.000000,0.000000,0.000000,0.000000,0.00,0.02,0.00,0.000000,0.02,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.080000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00
4,Chinatown,0.00,0.020000,0.02,0.00,0.000000,0.080000,0.000000,0.000000,0.02,0.000000,0.00,0.00000,0.500000,0.00,0.180000,0.000000,0.000000,0.00,0.00,0.02,0.000000,0.020000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.02,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.02,0.02,0.020000,0.000000,0.00
5,Civic Center,0.00,0.000000,0.02,0.00,0.000000,0.100000,0.020000,0.000000,0.00,0.000000,0.04,0.00000,0.500000,0.00,0.140000,0.000000,0.000000,0.00,0.00,0.02,0.000000,0.020000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.02,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.02,0.04,0.020000,0.000000,0.00
6,Clinton,0.00,0.020000,0.00,0.00,0.000000,0.120000,0.020000,0.000000,0.00,0.000000,0.00,0.00000,0.360000,0.00,0.260000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.020000,0.040000,0.00000,0.000000,0.000000,0.02,0.000

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Café,Coffee Shop,Bakery,Sandwich Place,Deli / Bodega,Bubble Tea Shop,Restaurant,Juice Bar,Italian Restaurant,Pizza Place
1,Carnegie Hill,Café,Coffee Shop,Bakery,French Restaurant,Bagel Shop,Salad Place,Gym / Fitness Center,Donut Shop,Pharmacy,Cocktail Bar
2,Central Harlem,Café,Coffee Shop,Ethiopian Restaurant,American Restaurant,Dessert Shop,Bar,Cafeteria,Donut Shop,Deli / Bodega,Diner
3,Chelsea,Café,Coffee Shop,Bakery,Sandwich Place,Diner,Cooking School,New American Restaurant,Japanese Restaurant,Italian Restaurant,Dessert Shop
4,Chinatown,Café,Coffee Shop,Bakery,French Restaurant,Internet Cafe,Dessert Shop,Cupcake Shop,Bookstore,Furniture / Home Store,Swiss Restaurant


In [30]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 3, 2, 2, 2, 4, 1, 0, 2], dtype=int32)

In [31]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Café,Bakery,Wine Shop,Donut Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Ethiopian Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,2,Café,Coffee Shop,Bakery,French Restaurant,Internet Cafe,Dessert Shop,Cupcake Shop,Bookstore,Furniture / Home Store,Swiss Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Café,Pet Café,Sandwich Place,Wine Shop,Dessert Shop,Cooking School,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Diner
3,Manhattan,Inwood,40.867684,-73.921210,1,Café,Coffee Shop,Seafood Restaurant,Diner,Cooking School,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Donut Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Café,Coffee Shop,American Restaurant,Cafeteria,Ethiopian Restaurant,Wine Shop,Diner,Cupcake Shop,Deli / Bodega,Dessert Shop


In [32]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Greenwich Village,Café,Coffee Shop,Bakery,French Restaurant,Italian Restaurant,Cupcake Shop,Cocktail Bar,Mediterranean Restaurant,Cafeteria,Furniture / Home Store
19,East Village,Café,Coffee Shop,Bakery,French Restaurant,Sandwich Place,Bookstore,Tea Room,Australian Restaurant,Sushi Restaurant,Korean Restaurant
20,Lower East Side,Café,Coffee Shop,Bakery,Bookstore,Tea Room,American Restaurant,Pet Service,Latin American Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant
21,Tribeca,Café,Coffee Shop,Bakery,French Restaurant,Dessert Shop,Cupcake Shop,Bubble Tea Shop,Italian Restaurant,Cocktail Bar,Mediterranean Restaurant
22,Little Italy,Café,Coffee Shop,Bakery,French Restaurant,Australian Restaurant,Gaming Cafe,Cupcake Shop,Cafeteria,Bookstore,Furniture / Home Store
23,Soho,Café,Coffee Shop,Bakery,French Restaurant,Cupcake Shop,Australian Restaurant,Italian Restaurant,Cocktail Bar,Mediterranean Restaurant,Cafeteria
24,West Village,Café,Coffee Shop,Bakery,Diner,American Restaurant,Dessert Shop,Cupcake Shop,Mediterranean Restaurant,New American Restaurant,Cafeteria
31,Noho,Café,Coffee Shop,Bakery,Australian Restaurant,French Restaurant,Gaming Cafe,Cupcake Shop,Internet Cafe,Hotel Bar,Bookstore
34,Sutton Place,Café,Coffee Shop,Bakery,Sandwich Place,Tea Room,Dessert Shop,French Restaurant,Juice Bar,Bar,Scandinavian Restaurant
38,Flatiron,Café,Coffee Shop,Bakery,Sandwich Place,American Restaurant,Australian Restaurant,Food Court,Hotel Bar,Italian Restaurant,Japanese Restaurant


In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Café,Bakery,Wine Shop,Donut Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Ethiopian Restaurant
2,Washington Heights,Café,Pet Café,Sandwich Place,Wine Shop,Dessert Shop,Cooking School,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Diner
3,Inwood,Café,Coffee Shop,Seafood Restaurant,Diner,Cooking School,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dessert Shop,Donut Shop
7,East Harlem,Café,Coffee Shop,Bakery,American Restaurant,Mexican Restaurant,Bagel Shop,Sandwich Place,Diner,Cupcake Shop,Deli / Bodega


In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Café,Coffee Shop,Bakery,French Restaurant,Internet Cafe,Dessert Shop,Cupcake Shop,Bookstore,Furniture / Home Store,Swiss Restaurant
8,Upper East Side,Café,Coffee Shop,Bakery,French Restaurant,Vegetarian / Vegan Restaurant,Salad Place,Dessert Shop,Pharmacy,Italian Restaurant,Donut Shop
9,Yorkville,Café,Coffee Shop,Bakery,French Restaurant,Vegetarian / Vegan Restaurant,Bagel Shop,Pharmacy,Gym / Fitness Center,Salad Place,Deli / Bodega
10,Lenox Hill,Café,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Dessert Shop,Deli / Bodega,Donut Shop,Italian Restaurant,Diner,New American Restaurant
12,Upper West Side,Café,Bakery,Coffee Shop,Diner,Convenience Store,Kosher Restaurant,Juice Bar,Italian Restaurant,Restaurant,Food Stand
13,Lincoln Square,Café,Bakery,Coffee Shop,Sandwich Place,Diner,Wine Shop,New American Restaurant,American Restaurant,Cuban Restaurant,Dessert Shop
16,Murray Hill,Café,Coffee Shop,Bakery,Sandwich Place,American Restaurant,Food Court,Deli / Bodega,Bar,Diner,Bookstore
17,Chelsea,Café,Coffee Shop,Bakery,Sandwich Place,Diner,Cooking School,New American Restaurant,Japanese Restaurant,Italian Restaurant,Dessert Shop
27,Gramercy,Café,Coffee Shop,Bakery,Sandwich Place,Italian Restaurant,Hotel Bar,Sushi Restaurant,Australian Restaurant,Flower Shop,Food & Drink Shop
28,Battery Park City,Café,Coffee Shop,Bakery,Sandwich Place,Deli / Bodega,Bubble Tea Shop,Restaurant,Juice Bar,Italian Restaurant,Pizza Place


In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Café,Coffee Shop,American Restaurant,Cafeteria,Ethiopian Restaurant,Wine Shop,Diner,Cupcake Shop,Deli / Bodega,Dessert Shop
5,Manhattanville,Café,Coffee Shop,College Cafeteria,American Restaurant,Bar,Donut Shop,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner
6,Central Harlem,Café,Coffee Shop,Ethiopian Restaurant,American Restaurant,Dessert Shop,Bar,Cafeteria,Donut Shop,Deli / Bodega,Diner
11,Roosevelt Island,Café,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,French Restaurant,Diner,Donut Shop,Flower Shop,Ethiopian Restaurant,Convenience Store
25,Manhattan Valley,Café,Convenience Store,Dessert Shop,French Restaurant,Coffee Shop,Kosher Restaurant,Deli / Bodega,Food Truck,Bakery,Donut Shop
26,Morningside Heights,Café,Coffee Shop,Dessert Shop,College Cafeteria,Bar,Deli / Bodega,French Restaurant,Bakery,Cupcake Shop,Food Truck
37,Stuyvesant Town,Café,Coffee Shop,Bakery,Beer Garden,Korean Restaurant,Jazz Club,Breakfast Spot,Diner,Russian Restaurant,Sandwich Place


In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Clinton,Café,Coffee Shop,Bakery,Sandwich Place,Donut Shop,Food Court,Japanese Restaurant,Diner,Wine Bar,American Restaurant
15,Midtown,Café,Coffee Shop,Bakery,Sandwich Place,Donut Shop,Diner,Bookstore,Food Court,Deli / Bodega,French Restaurant
35,Turtle Bay,Café,Coffee Shop,Bakery,Sandwich Place,Deli / Bodega,Italian Restaurant,Juice Bar,Bagel Shop,Scandinavian Restaurant,Japanese Restaurant
36,Tudor City,Café,Coffee Shop,Bakery,Sandwich Place,Deli / Bodega,Japanese Restaurant,Donut Shop,Italian Restaurant,Juice Bar,Bagel Shop
